In [ ]:
import numpy as np
import ipywidgets as ipw

from bokeh.io import push_notebook, output_notebook, show
from bokeh.layouts import row, column
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
output_notebook()

from ipywidgets import interact

from collections import OrderedDict
old_settings = np.seterr(over = 'ignore') #Ignore warnings about overflow data points

In [ ]:
kz = np.linspace(-np.pi, np.pi);
Ez = -2*np.cos(kz)-3.0;
B = 0.5; # Magnetic field
hbar = 1.0;
e = 1; # Electron charge
m = 1; # (free) Electron mass
c = 1; # Speed of light
wc = e*B/(m*c); # Cyclotron frequency

In [ ]:
# Define the Fermi surface in zero magnetic field
freeE = figure(plot_height=350, plot_width=400, title="Dispersion relationship of a free electron",
              tools="pan,reset,save,wheel_zoom", x_range = [-1.1*np.amin(Ez), 1.1*np.amin(Ez)],
              y_range = [np.amin(kz), np.amax(kz)]);
source = ColumnDataSource(data = {'xVal':Ez, 'yVal':kz, 'xxVal':-Ez});
freeE.line('xVal', 'yVal', source = source, line_width=2, line_alpha=1);
freeE.line('xxVal', 'yVal', source = source, line_width=2, line_alpha=1);

In [ ]:
dE = hbar*wc;
dA = 2*m*np.pi*wc/hbar;
A = np.arange(0, 31*dA, dA);
kk = np.sqrt(A)/np.pi;
kF = np.array([-kk,kk]);
kF.flatten()
Ef = -2*np.cos(kF)-3.0;
gExy = m/(np.pi*hbar**2);
source2 = ColumnDataSource(data = {'xVal':Ef, 'yVal': kF, 'xxVal': -Ef});
freeE.circle('xVal','yVal', source = source2, line_width = 2, line_alpha = 0.2);
freeE.circle('xxVal','yVal', source = source2, line_width = 2, line_alpha = 0.2);
freeE.xaxis.axis_label = 'kx & ky';
freeE.yaxis.axis_label = 'kz'
show(freeE)

In [ ]:
# Plot the histogram of states per energy interval
dEf = np.diff(Ef);
bWidth = np.amax(np.abs(dEf)); # Define the bin width of the histogram so minimum it contains one kxy tube
histoN = (2*np.amax(kz)/bWidth).astype(int); # calculate the number of bars in the histogram
kCount = np.empty(histoN, dtype = np.int8); # Declare an array to store histogram information
gEs = np.linspace(kz.min(),kz.max(), histoN);
zero = np.zeros(histoN);
for idx in range(histoN-1):
    kCount[idx] = len(kF[ (kF > gEs[idx]) & (kF <= gEs[idx+1]) ]);
histo = figure(plot_height=350, plot_width=300, title="allowed states per Energy interval",
              tools="pan,reset,save,wheel_zoom", y_range = [1.1*np.amin(gEs), 1.1*np.amax(gEs)],
              x_range = [0, 1.1*np.amax(kCount)]);
histo.xaxis[0].axis_label='Number of states';
histo.yaxis[0].axis_label='Energy (eV)';
source1 = ColumnDataSource(data = {'bottom': gEs, 'top': gEs + bWidth, 'right': kCount*gExy, 'left': zero});
histo.quad(bottom = 'bottom', top = 'top', right = 'right', left = 'left', source = source1,
           fill_color="navy", line_color="white", alpha=1);
show(histo)